## Modified greedy search 

Credit to @crodoc for the original version of this notebook that I've modified and extended

In [1]:
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

root_dir = "competition_data"

In [2]:
puzzles_df = pd.read_csv(os.path.join(root_dir, 'puzzles.csv'))
puzzles_df

,id,puzzle_type,solution_state,initial_state,num_wildcards
0,0,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;D;A;E;B;A;B;C;A;C;A;D;C;D;F;F;F;E;E;B;F;B;C,0
1,1,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;C;B;B;E;F;A;F;D;B;F;F;E;B;D;A;A;C;D;C;E;A;C,0
2,2,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;F;C;C;F;A;D;D;B;B;A;F;E;B;C;A;A;B;D;F;E;E;C;D,0
3,3,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,A;C;E;C;F;D;E;D;A;A;F;A;B;D;B;F;E;D;B;F;B;C;C;E,0
4,4,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;D;E;D;A;E;F;B;A;C;F;D;F;D;C;A;F;B;C;C;B;E;B;A,0
...,...,...,...,...,...
393,393,globe_3/33,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,D;D;L;A;P;E;R;U;U;C;S;R;J;B;E;G;O;J;F;Q;R;E;D;...,0
394,394,globe_3/33,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,V;L;N;G;B;V;R;E;H;A;K;S;I;N;G;E;V;C;L;G;S;M;P;...,0
395,395,globe_3/33,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N12;N219;N227;N198;N4;N208;N214;N245;N56;N55;N...,0
396,396,globe_8/25,A;A;A;A;A;D;D;D;D;D;G;G;G;G;G;J;J;J;J;J;M;M;M;...,V;P;F;L;P;X;O;A;J;b;V;Y;D;Y;C;X;J;F;U;G;d;L;b;...,0


In [3]:
puzzle_info_df = pd.read_csv(os.path.join(root_dir, 'puzzle_info.csv'))
puzzle_info_df

,puzzle_type,allowed_moves
0,cube_2/2/2,"{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,..."
1,cube_3/3/3,"{'f0': [0, 1, 2, 3, 4, 5, 44, 41, 38, 15, 12, ..."
2,cube_4/4/4,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
3,cube_5/5/5,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
4,cube_6/6/6,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
5,cube_7/7/7,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
6,cube_8/8/8,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
7,cube_9/9/9,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
8,cube_10/10/10,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
9,cube_19/19/19,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."


In [4]:
allowed_moves = {}

for _, row in puzzle_info_df.iterrows():
    allowed_moves[row['puzzle_type']] = eval(row['allowed_moves'])

# examples of parsed allowed moves
# cube puzzles
print("cube_2/2/2:")
for move_name, move_permutation in allowed_moves['cube_2/2/2'].items():
    print(move_name, move_permutation)
    
# wreath puzzles
print("wreath_12/12:")
for move_name, move_permutation in allowed_moves['wreath_12/12'].items():
    print(move_name, move_permutation)

# globe puzzles
print("globe_3/4:")
for move_name, move_permutation in allowed_moves['globe_3/4'].items():
    print(move_name, move_permutation)


cube_2/2/2:
f0 [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11, 12, 13, 14, 15, 16, 20, 18, 21, 10, 8, 22, 23]
f1 [18, 16, 2, 3, 4, 5, 6, 7, 8, 0, 10, 1, 13, 15, 12, 14, 22, 17, 23, 19, 20, 21, 11, 9]
r0 [0, 5, 2, 7, 4, 21, 6, 23, 10, 8, 11, 9, 3, 13, 1, 15, 16, 17, 18, 19, 20, 14, 22, 12]
r1 [4, 1, 6, 3, 20, 5, 22, 7, 8, 9, 10, 11, 12, 2, 14, 0, 17, 19, 16, 18, 15, 21, 13, 23]
d0 [0, 1, 2, 3, 4, 5, 18, 19, 8, 9, 6, 7, 12, 13, 10, 11, 16, 17, 14, 15, 22, 20, 23, 21]
d1 [1, 3, 0, 2, 16, 17, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12, 13, 18, 19, 20, 21, 22, 23]
wreath_12/12:
l [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
r [12, 1, 2, 19, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 3, 20, 21, 0]
globe_3/4:
r0 [1, 2, 3, 4, 5, 6, 7, 0, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
r1 [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 8, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
r2 [0, 1, 2, 3, 4

### Some quick notes about puzzle structure
- Cube_x/x/x puzzles have a set of moves {f0, f1, ..., f(x-1), r0, r1, ..., r(x-1), d0, d1, ..., d(x-1)}
- Wreath_x/x puzzles have a set of moves {l, r}
- Globe_x/y puzzles have a set of moves {r0, r1, ..., r(x), f0, f1, ..., f(2y-1)}

In [5]:
submission_df = pd.read_csv(os.path.join(root_dir, 'sample_submission.csv'))
submission_df

,id,moves
0,0,r1.-f1
1,1,f1.d0.-r0.-f1.-d0.-f1.d0.-r0.f0.-f1.-r0.f1.-d1...
2,2,f1.d0.-d1.r0.-d1.-f0.f1.-r0.-f0.-r1.-f0.r0.-d0...
3,3,-f0.-r0.-f0.-d0.-f0.f1.r0.-d1.-r0.-r1.-r0.-f1....
4,4,d1.-f1.d1.r1.-f0.d1.-d0.-r1.d1.d1.-f1.d1.-d0.-...
...,...,...
393,393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394,394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395,395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....
396,396,f25.-f29.f46.f49.-f8.f27.f26.-f20.f2.-f20.f6.f...


In [6]:
def move_state(state:np.ndarray, move_key:str, moves_pool:dict[str, np.ndarray]) -> np.ndarray:
    
    # if '-' in move_key:
    #     move_key = move_key[1:]
    #     rev = True
    # else:
    #     rev = False

    move = moves_pool[move_key]

    # if rev:
    #     # Reverse the permutation defined by move using np.argsort()
    #     state = state[np.argsort(move)]
    # else:
    #     state = state[move]
    
    return state[move]

In [7]:
# Scratchpad code to test numpy arrays for state definition
first_puzzle = puzzles_df.iloc[0]
puzzle_type = first_puzzle['puzzle_type']
initial_state_str = first_puzzle['initial_state']
solution_state_str = first_puzzle['solution_state']

initial_state = np.array(initial_state_str.split(';'))
solution_state = np.array(solution_state_str.split(';'))

print(initial_state)
print(solution_state)

moves_pool = allowed_moves[puzzle_type]
print(moves_pool.keys())
# move_keylist = list(moves_pool.keys())
# for move_key in move_keylist:
#     augmented_move_key = "-" + move_key
#     moves_pool[augmented_move_key] = np.argsort(moves_pool[move_key])
# print(moves_pool.keys())


state = initial_state.copy()
move_index = "f0"
state = state[moves_pool[move_index]]
print(moves_pool[move_index])
print(np.argsort(moves_pool[move_index]))
print(state)
state = state[np.argsort(moves_pool[move_index])]
print(state)

['D' 'E' 'D' 'A' 'E' 'B' 'A' 'B' 'C' 'A' 'C' 'A' 'D' 'C' 'D' 'F' 'F' 'F'
 'E' 'E' 'B' 'F' 'B' 'C']
['A' 'A' 'A' 'A' 'B' 'B' 'B' 'B' 'C' 'C' 'C' 'C' 'D' 'D' 'D' 'D' 'E' 'E'
 'E' 'E' 'F' 'F' 'F' 'F']
dict_keys(['f0', 'f1', 'r0', 'r1', 'd0', 'd1'])
[0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11, 12, 13, 14, 15, 16, 20, 18, 21, 10, 8, 22, 23]
[ 0  1  8 10  5  7  4  6 21  9 20 11 12 13 14 15 16  3 18  2 17 19 22 23]
['D' 'E' 'E' 'F' 'A' 'E' 'B' 'B' 'D' 'A' 'A' 'A' 'D' 'C' 'D' 'F' 'F' 'B'
 'E' 'F' 'C' 'C' 'B' 'C']
['D' 'E' 'D' 'A' 'E' 'B' 'A' 'B' 'C' 'A' 'C' 'A' 'D' 'C' 'D' 'F' 'F' 'F'
 'E' 'E' 'B' 'F' 'B' 'C']


In [11]:
def solve_puzzle(puzzle_id):
    
    current_puzzle = puzzles_df.loc[puzzle_id]
    puzzle_type = current_puzzle['puzzle_type']
    initial_state = current_puzzle['initial_state']
    solution_state = current_puzzle['solution_state']
    
    # Pull the moves from the submission_df
    move_keys = submission_df.loc[puzzle_id]['moves'].split('.')
    moves_pool = allowed_moves[puzzle_type]

    # Add the reverse moves to the moves_pool
    # move_keylist = list(moves_pool.keys())
    # for move_key in move_keylist:
    #     augmented_move_key = "-" + move_key
    #     moves_pool[augmented_move_key] = np.argsort(moves_pool[move_key])
    # print(moves_pool.keys())

    state = np.array(initial_state.split(';'))
    
    state_list = []
    state_list.append(state)
    
    # Reconstruct the state history
    for move_key in move_keys:
        state = move_state(state, move_key, moves_pool)
        state_list.append(state)
    
    state_to_idx = {}
    for idx in range(len(state_list)):
        state_to_idx[';'.join(state_list[idx])] = idx
    
    res = []
    idx = 0
    
    # Look for shortcuts in the state history by testing possible moves from each historical state
    # and looking for matches
    for curr_idx in tqdm(list(range(len(state_list)-1))):
        
        if curr_idx != idx:
            continue
        
        state = state_list[idx]
        
        new_idx = -1
        new_move = ''
        
        for move in moves_pool:
            for reversed_move in ['', '-']:
                move = reversed_move + move
                
                new_state = ';'.join(move_state(state, move, moves_pool))
                
                if new_state in state_to_idx:
                    tmp_idx = state_to_idx[new_state]

                    if tmp_idx > new_idx:
                        new_idx = tmp_idx
                        new_move = move
        
        idx = new_idx
        res.append(new_move)
            
    print('PUZZLE_ID:', puzzle_id)
    print('MOVES BEFORE:', len(state_list)-1)
    print('MOVES AFTER:', len(res))
    print()
    
    return '.'.join(res)

In [12]:
for i in range(396,398):
    
    if i in [281, 282, 283]:
        continue
        
    submission_df.loc[i, 'moves'] = solve_puzzle(i)

  7%|▋         | 1839/26402 [00:50<11:08, 36.73it/s]


KeyboardInterrupt: 

In [ ]:
submission_df.to_csv('submission.csv', index=False)
submission_df